In [1]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

グラフの作成に必要なライブラリ等の準備

In [2]:
!pip install japanize-matplotlib

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import japanize_matplotlib 

In [4]:
!pip install plotly==5.1.0

In [5]:
!pip install -U kaleido

In [6]:
# Google Colab. やJupyter Lab.でplotlyを使用するための準備
import plotly.io as pio
from plotly.subplots import make_subplots
pio.renderers.default = "colab"

from plotly.offline import iplot, plot, download_plotlyjs, init_notebook_mode
import plotly.graph_objs as go

In [ ]:
!pip install fpdf

# 「IPA」フォントをインストール
!apt-get -y install fonts-ipafont-gothic
# matplotlibのキャッシュをクリア
!rm /root/.cache/matplotlib/fontlist-v300.json

plt.rcParams['font.family']
plt.rcParams['font.family'] = 'IPAexGothic'

各データファイルの取得（Driveからファイルを取得する場合）

In [ ]:
# ブラウザでフォルダを表示するとURLのfolders以降がフォルダのIDになっている
# https://drive.google.com/drive/folders/<ID>
dir_id = "取得対象のファイルのある場所"

# フォルダ内にあるファイル名とファイルIDを表示する
file_list_input = drive.ListFile({'q': "'%s' in parents and trashed=false" % dir_id}).GetList()

file_list_input

In [ ]:
file_dict = {}
for f in file_list_input:
  if(f["title"] in "横浜の平均湿度.csv"):
    file_dict["横浜の平均湿度.csv"] = f["id"]
  elif(f["title"] in "横浜の平均気温.csv"):
    file_dict["横浜の平均気温.csv"] = f["id"]

file_dict

In [10]:
# 上記の方法で調べたファイルのID
file_id = file_dict["横浜の平均湿度.csv"]
drive_file = drive.CreateFile({'id': file_id})

# 横浜の平均湿度データファイルの取得
drive_file.GetContentFile("横浜の平均湿度.csv")

average_humidity = pd.read_csv('横浜の平均湿度.csv', encoding="shift-jis", header=1)
print(average_humidity.head())

        年月日  平均湿度(％)
0       NaN      NaN
1  2021/1/1     42.0
2  2021/1/2     53.0
3  2021/1/3     51.0
4  2021/1/4     53.0


In [11]:
file_id = file_dict["横浜の平均気温.csv"]
drive_file = drive.CreateFile({'id': file_id})

# 2021年の平均気温データファイルの取得
drive_file.GetContentFile("横浜の平均気温.csv")

average_temperature = pd.read_csv('横浜の平均気温.csv', encoding="shift-jis", header=1)
print(average_temperature.head())

       DATE  平均気温(℃)
0       NaN      NaN
1  2021/1/1      6.0
2  2021/1/2      6.5
3  2021/1/3      5.0
4  2021/1/4      6.7


In [12]:
average_temperature.dropna(inplace=True)
print(average_temperature.head())

       DATE  平均気温(℃)
1  2021/1/1      6.0
2  2021/1/2      6.5
3  2021/1/3      5.0
4  2021/1/4      6.7
5  2021/1/5      6.9


In [13]:
# 試しにDATEをindexに設定する
a_temp = average_temperature.set_index("DATE")
print(a_temp)

            平均気温(℃)
DATE               
2021/1/1        6.0
2021/1/2        6.5
2021/1/3        5.0
2021/1/4        6.7
2021/1/5        6.9
...             ...
2021/12/27      3.6
2021/12/28      4.5
2021/12/29      5.8
2021/12/30      7.9
2021/12/31      3.6

[365 rows x 1 columns]


In [14]:
# renameできない
a_temp.rename(columns={"DATE": "年月日"},
          inplace=True)
print(a_temp)

            平均気温(℃)
DATE               
2021/1/1        6.0
2021/1/2        6.5
2021/1/3        5.0
2021/1/4        6.7
2021/1/5        6.9
...             ...
2021/12/27      3.6
2021/12/28      4.5
2021/12/29      5.8
2021/12/30      7.9
2021/12/31      3.6

[365 rows x 1 columns]


In [15]:
# 一度インデックスをリセットしてからrenameする
a_temp =average_temperature.reset_index(drop=True)
a_temp.rename(columns={"DATE": "年月日"},
          inplace=True)
print(a_temp)

            年月日  平均気温(℃)
0      2021/1/1      6.0
1      2021/1/2      6.5
2      2021/1/3      5.0
3      2021/1/4      6.7
4      2021/1/5      6.9
..          ...      ...
360  2021/12/27      3.6
361  2021/12/28      4.5
362  2021/12/29      5.8
363  2021/12/30      7.9
364  2021/12/31      3.6

[365 rows x 2 columns]


In [16]:
average_humidity.dropna(inplace=True)

In [17]:
df = pd.merge(a_temp, average_humidity, on="年月日")
df.info()
print(df)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 364 entries, 0 to 363
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   年月日      364 non-null    object 
 1   平均気温(℃)  364 non-null    float64
 2   平均湿度(％)  364 non-null    float64
dtypes: float64(2), object(1)
memory usage: 11.4+ KB
            年月日  平均気温(℃)  平均湿度(％)
0      2021/1/1      6.0     42.0
1      2021/1/2      6.5     53.0
2      2021/1/3      5.0     51.0
3      2021/1/4      6.7     53.0
4      2021/1/5      6.9     61.0
..          ...      ...      ...
359  2021/12/27      3.6     37.0
360  2021/12/28      4.5     45.0
361  2021/12/29      5.8     53.0
362  2021/12/30      7.9     52.0
363  2021/12/31      3.6     52.0

[364 rows x 3 columns]


In [18]:
jp_font = {'fontname':'IPAGothic'} # 日本語フォント設定を作成

In [21]:
import matplotlib.font_manager as font_manager

# 2つ目の軸を表示するか否かを設定
fig = make_subplots(specs=[[{"secondary_y": True}]])

# 第一軸のグラフ
trace1 = fig.add_trace(go.Bar(
            x=df["年月日"],
            y=df["平均気温(℃)"],
            name ="平均気温"
       ))

# 第二軸のグラフ
trace2 = fig.add_trace(go.Line(
            x=df["年月日"],
            y=df["平均湿度(％)"],
            name ="平均湿度"),
       secondary_y=True)

data = [trace1, trace2]

# グラフのタイトルと各軸のタイトル設定
fig.update_layout(
    title="<b>横浜の平均気温と平均湿度の関係</b>"
)
fig.update_yaxes(title_text="<b>平均湿度(％)</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>平均気温(℃)</b>", secondary_y=True)
fig.update_xaxes(title="<b>年月日</b>")

# このまま出力すると文字化けしてしまうので日本語のフォントを設定する
font = font_manager.FontProperties(
    family="IPAGothic",
    size=30
)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




使用環境

In [24]:
%pip freeze

absl-py==1.0.0
alabaster==0.7.12
albumentations==0.1.12
altair==4.2.0
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arviz==0.11.4
astor==0.8.1
astropy==4.3.1
astunparse==1.6.3
atari-py==0.2.9
atomicwrites==1.4.0
attrs==21.4.0
audioread==2.1.9
autograd==1.3
Babel==2.9.1
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==4.1.0
blis==0.4.1
bokeh==2.3.3
Bottleneck==1.3.4
branca==0.4.2
bs4==0.0.1
CacheControl==0.12.10
cached-property==1.5.2
cachetools==4.2.4
catalogue==1.0.0
certifi==2021.10.8
cffi==1.15.0
cftime==1.6.0
chardet==3.0.4
charset-normalizer==2.0.12
click==7.1.2
cloudpickle==1.3.0
cmake==3.12.0
cmdstanpy==0.9.5
colorcet==3.0.0
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.4.0
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.17.3
cvxopt==1.2.7
cvxpy==1.0.31
cycler==0.11.0
cymem==2.0.6
Cython==0.29.28
daft==0.0.4
dask==2.12.0
datascience==0.10.6
debugpy==1.0.0
decorator==4.4.2
defusedxml==0.7.1
descartes==1.1.0
dill==0.3.4
distributed==

In [25]:
! python -V

Python 3.7.13
